In [1]:
import pandas as pd
import numpy as np
import os 

In [2]:
dataset_list = ['MUTAG','PTC_MR','ENZYMES','PROTEINS','DD']
method_list = ['HKS','WKS']
column_list = ['dataset','method','C','gamma','w','sinkhorn','acc','std']


In [3]:
data = []
for dataset in dataset_list:
    for method in method_list:
        Path = './results/'+dataset+'/'+method+'/'
        list_of_files = sorted( filter( lambda x: os.path.isfile(os.path.join(Path, x)),
                                os.listdir(Path) ) )
        result_files = [pd.read_csv(Path+i) for i in list_of_files]
        means = np.array([np.mean(file["accuracy"]) for file in result_files])
        stds = np.array([np.std(file["accuracy"]) for file in result_files])
        best_df = result_files[np.argmax(means)].iloc[1]
        data_unit = [dataset,method,best_df['C'],best_df['gamma'],best_df['w'],False,np.max(means),stds[np.argmax(means)]]
        if dataset == 'DD':
            data_unit[-3] = True
        data.append(data_unit)

In [4]:
df = pd.DataFrame(data,columns=column_list)

In [5]:
df

,dataset,method,C,gamma,w,sinkhorn,acc,std
0,MUTAG,HKS,10.0,1.00,0.30,False,0.898830,0.043922
1,MUTAG,WKS,1000.0,0.10,0.60,False,0.925146,0.036030
2,PTC_MR,HKS,1000.0,1.00,0.35,False,0.619244,0.045103
3,PTC_MR,WKS,1.0,10.00,0.60,False,0.665798,0.049955
4,ENZYMES,HKS,100.0,0.01,0.55,False,0.693333,0.050111
5,ENZYMES,WKS,100.0,0.01,0.55,False,0.693333,0.044845
6,PROTEINS,HKS,1.0,1.00,0.25,False,0.775201,0.030743
7,PROTEINS,WKS,1.0,1.00,0.10,False,0.767109,0.031594
8,DD,HKS,1000.0,1.00,0.45,True,0.785959,0.030707
9,DD,WKS,1000.0,1.00,0.35,True,0.788622,0.030018


In [6]:
df.to_csv('best.csv')

In [9]:
hks_acc = [acc for acc in df[df['method']=='WKS']['acc']]
hks_std = [std for std in df[df['method']=='WKS']['std']]
hks_res = ['{:2.2f}+-{:2.2f}%'.format(hks_acc[i]*100,(hks_std[i]/np.sqrt(10))*100) for i in range(len(hks_acc))]
hks_df = pd.DataFrame([hks_acc,hks_std,hks_res],columns=dataset_list)

In [10]:
hks_df

,MUTAG,PTC_MR,ENZYMES,PROTEINS,DD
0,0.925146,0.665798,0.693333,0.767109,0.788622
1,0.03603,0.049955,0.044845,0.031594,0.030018
2,92.51+-1.14%,66.58+-1.58%,69.33+-1.42%,76.71+-1.00%,78.86+-0.95%
